In [17]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


True
Tesla T4


In [18]:
!ls


drive  sample_data


In [19]:
!ls drive


MyDrive


In [20]:
!ls /content


drive  sample_data


In [21]:
!ls /content/drive


MyDrive


In [22]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
!ls /content


drive  sample_data


In [24]:
!ls /content/drive


MyDrive


In [25]:
!ls /content/drive/MyDrive/CAT_ROBOFLOW


ls: cannot access '/content/drive/MyDrive/CAT_ROBOFLOW': No such file or directory


In [26]:
!ls /content/drive/MyDrive/CAT_ROBOFLOW/train


ls: cannot access '/content/drive/MyDrive/CAT_ROBOFLOW/train': No such file or directory


In [27]:
!ls /content/drive/MyDrive/CAT_ROBOFLOW/valid


ls: cannot access '/content/drive/MyDrive/CAT_ROBOFLOW/valid': No such file or directory


In [28]:
TRAIN_DIR = "/content/drive/MyDrive/CAT_ROBOFLOW/train"
VAL_DIR   = "/content/drive/MyDrive/CAT_ROBOFLOW/valid"


In [29]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

IMG_SIZE = 224
BATCH_SIZE = 32

train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

val_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [30]:
train_dataset = datasets.ImageFolder(TRAIN_DIR, transform=train_transforms)
val_dataset   = datasets.ImageFolder(VAL_DIR, transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

print("Classes:", train_dataset.classes)
print("Number of classes:", len(train_dataset.classes))


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/CAT_ROBOFLOW/train'

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


In [ ]:
from torchvision import models
import torch.nn as nn

model = models.resnet50(
    weights=models.ResNet50_Weights.IMAGENET1K_V1
)


In [ ]:
for param in model.parameters():
    param.requires_grad = False


In [ ]:
num_classes = len(train_dataset.classes)

model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 512),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(512, num_classes)
)


In [ ]:
model = model.to(device)


In [ ]:
criterion = nn.CrossEntropyLoss()


In [ ]:
import torch.optim as optim

optimizer = optim.AdamW(
    model.fc.parameters(),
    lr=3e-4,
    weight_decay=1e-4
)


In [ ]:
def train_one_epoch(model, loader):
    model.train()
    running_loss = 0
    correct = 0
    total = 0

    for images, labels in loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return running_loss / len(loader), correct / total


In [ ]:
def validate(model, loader):
    model.eval()
    correct = 0
    total = 0
    val_loss = 0

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return val_loss / len(loader), correct / total


In [ ]:
EPOCHS = 10
best_val_acc = 0

for epoch in range(EPOCHS):
    train_loss, train_acc = train_one_epoch(model, train_loader)
    val_loss, val_acc = validate(model, val_loader)

    print(f"Epoch {epoch+1}/{EPOCHS}")
    print(f"Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_resnet_head.pth")


In [ ]:
for param in model.parameters():
    param.requires_grad = True


In [ ]:
optimizer = optim.AdamW(
    model.parameters(),
    lr=1e-4,
    weight_decay=1e-5
)


In [ ]:
EPOCHS_FINE = 15

for epoch in range(EPOCHS_FINE):
    train_loss, train_acc = train_one_epoch(model, train_loader)
    val_loss, val_acc = validate(model, val_loader)

    print(f"[Fine-Tune] Epoch {epoch+1}/{EPOCHS_FINE}")
    print(f"Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_resnet_finetuned.pth")


In [ ]:
EPOCHS = 20
best_val_acc = 0

for epoch in range(EPOCHS):
    train_loss, train_acc = train_one_epoch(model, train_loader)
    val_loss, val_acc = validate(model, val_loader)

    print(f"Epoch {epoch+1}/{EPOCHS}")
    print(f"Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_resnet_head.pth")


In [ ]:
EPOCHS = 5
best_val_acc = 0

for epoch in range(EPOCHS):
    train_loss, train_acc = train_one_epoch(model, train_loader)
    val_loss, val_acc = validate(model, val_loader)

    print(f"Epoch {epoch+1}/{EPOCHS}")
    print(f"Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_resnet_head.pth")


In [ ]:
for param in model.parameters():
    param.requires_grad = True


In [ ]:
optimizer = optim.AdamW(
    model.parameters(),
    lr=1e-4,
    weight_decay=1e-5
)


In [ ]:
model.load_state_dict(torch.load("best_resnet_head.pth"))


In [ ]:
model = model.to(device)


In [ ]:
for param in model.parameters():
    param.requires_grad = True


In [ ]:
import torch.optim as optim

optimizer = optim.AdamW(
    model.parameters(),
    lr=1e-4,
    weight_decay=1e-5
)


In [ ]:
EPOCHS_FINE = 10
best_val_acc = 0

for epoch in range(EPOCHS_FINE):
    train_loss, train_acc = train_one_epoch(model, train_loader)
    val_loss, val_acc = validate(model, val_loader)

    print(f"[Fine-Tune] Epoch {epoch+1}/{EPOCHS_FINE}")
    print(f"Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(
            model.state_dict(),
            "/content/drive/MyDrive/CAT_ROBOFLOW/models/best_resnet_finetuned.pth"
        )


In [ ]:
!mkdir -p /content/drive/MyDrive/CAT_ROBOFLOW/models


In [ ]:
EPOCHS_FINE = 10
best_val_acc = 0

for epoch in range(EPOCHS_FINE):
    train_loss, train_acc = train_one_epoch(model, train_loader)
    val_loss, val_acc = validate(model, val_loader)

    print(f"[Fine-Tune] Epoch {epoch+1}/{EPOCHS_FINE}")
    print(f"Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(
            model.state_dict(),
            "/content/drive/MyDrive/CAT_ROBOFLOW/models/best_resnet_finetuned.pth"
        )


In [ ]:
!ls /content/drive/MyDrive/CAT_ROBOFLOW/models


In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
# Recreate ResNet50 with same classifier
num_classes = len(train_dataset.classes)

model = models.resnet50(
    weights=models.ResNet50_Weights.IMAGENET1K_V1
)

model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 512),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(512, num_classes)
)


In [ ]:
MODEL_PATH = "/content/drive/MyDrive/CAT_ROBOFLOW/models/best_resnet_finetuned.pth"

model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model = model.to(device)
model.eval()


In [ ]:
infer_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [ ]:
IMAGE_PATH = "/content/drive/MyDrive/test_image.jpg"


In [ ]:
!ls /content/drive/MyDrive/CAT_ROBOFLOW


In [ ]:
!ls /content/drive/MyDrive/CAT_ROBOFLOW/valid


In [ ]:
IMAGE_PATH = "/content/drive/MyDrive/CAT_ROBOFLOW/valid/Happy/img_023.jpg"



In [ ]:
!ls /content/drive/MyDrive/CAT_ROBOFLOW/valid



In [ ]:
!ls /content/drive/MyDrive/CAT_ROBOFLOW/valid/Happy | head


In [ ]:
IMAGE_PATH = "/content/drive/MyDrive/CAT_ROBOFLOW/valid/Happy/10_h_jpeg.rf.1ab362f340524f4490188b73e9551357.jpg"


In [ ]:
from PIL import Image

image = Image.open(IMAGE_PATH).convert("RGB")


In [ ]:
import matplotlib.pyplot as plt

plt.imshow(image)
plt.axis("off")


In [ ]:
IMAGE_PATH = "/content/drive/MyDrive/CAT_ROBOFLOW/valid/Happy/11_h_jpeg.rf.b43139316c03dc5d6b39a2888bebe618.jpg"

In [ ]:
from PIL import Image

image = Image.open(IMAGE_PATH).convert("RGB")

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(image)
plt.axis("off")

In [ ]:
image_tensor = infer_transform(image).unsqueeze(0).to(device)

with torch.no_grad():
    outputs = model(image_tensor)
    probs = torch.softmax(outputs, dim=1)
    pred_idx = torch.argmax(probs, dim=1).item()

pred_class = train_dataset.classes[pred_idx]
confidence = probs[0][pred_idx].item()

print("Predicted class:", pred_class)
print("Confidence:", round(confidence * 100, 2), "%")
